In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets==2.13.1
!pip install transformers
!pip install accelerate

In [ ]:
import datasets
import json
import torch
import pandas as pd
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, Dataset
from accelerate import Accelerator

In [ ]:
df = datasets.load_dataset("ai4bharat/IndicQA", "indicqa.hi")

In [ ]:
df = df["test"]

In [ ]:
df_hindi = pd.DataFrame(df)

In [ ]:
df_hindi.head(10)

In [ ]:
def buildprompt(data):
    prompt = {}
    prompt['text'] = "Given the context delimited by triple backticks ```{" + data['context'] + "}```, generate the following question answer pair. Question: {" + data['question'] + "} Answer: {" + data['answers']['text'][0] + "} Answer Start at index: {" + (str(data['answers']['answer_start'][0]) if data['answers']['answer_start'][0] else "") + "}" 
    return prompt

df_hindi['prompt'] = df_hindi.apply(buildprompt, axis=1)

result = df_hindi['prompt'].to_list()

In [ ]:
# result[:3]

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# !nvidia-smi

In [ ]:
checkpoint = "bigscience/mt0-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto")

In [ ]:
result = result[:100]

In [ ]:
with open('/kaggle/working/prompts.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

In [ ]:
# dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=result))

In [ ]:
# dataset

In [ ]:
# !pip install datasets==2.13.1

In [ ]:
dataset = load_dataset("json", data_files="/kaggle/working/prompts.json")

In [ ]:
# import datasets
# print(datasets.__version__)

In [ ]:
# dataset = load_dataset("json", data_files="/kaggle/working/prompts.json")

In [ ]:
dataset

In [ ]:
def prepare_train_data(data):
    # prompt + completion
    #text_input = data['prompt'] + ' ' + data['completion']
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(text_input, return_tensors='pt', padding='max_length', truncation=True, max_length=2050)
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

In [ ]:
train_dataset = dataset.map(prepare_train_data,batched=True, remove_columns=["text"])

In [ ]:
training_arguments = TrainingArguments(
    '/kaggle/working/HindiQA-mt0-small',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
)

In [ ]:
import transformers
data_collator = transformers.DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/HindiQA-mt0-small")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("./HindiQA-mt0-small")

In [ ]:
essay = "गणतंत्र दिवस को 26 जनवरी भी कहा जाता है जो हर साल पूरे भारत में मनाया जाता है. इसी दिन भारत को एक गणतांत्रिक देश घोषित किया गया था साथ ही आजादी के लंबे संघर्ष के बाद भारतीयों को अपनी कानूनी किताब ‘संविधान’ की प्राप्ति हुई थी। 15 अगस्त 1947 को भारत आजाद हुआ और इसके ढ़ाई साल बाद ये लोकतांत्रिक गणराज्य के रुप में स्थापित हुआ।आजादी के बाद एक ड्राफ्टिंग कमेटी को 28 अगस्त 1947 की मीटिंग में भारत के स्थायी संविधान का प्रारुप तैयार करने को कहा गया। 4 नवंबर 1947 को डॉ बी.आर.अंबेडकर की अध्यक्षता में भारतीय संविधान के प्रारुप को सदन में रखा गया।इसे पूरी तरह तैयार होने में लगभग तीन साल का समय लगा और आखिरकार इंतजार की घड़ी 26 जनवरी 1950 को इसको लागू होने के साथ ही खत्म हुई। साथ ही पूर्णं स्वराज की प्रतिज्ञा का भी सम्मान हुआ।भारत में गणतंत्र दिवस का दिन राष्ट्रीय अवकाश के रुप में मनाया जाता है जब इस महान दिन का उत्सव लोग अपने-अपने तरीके से मनाते है, जैसे- समाचार देखकर, स्कूल में भाषण के द्वारा या भारत की आजादी से संबंधित किसी प्रतियोगिता में भाग लेकर आदि।इस दिन भारतीय सरकार द्वारा नई दिल्ली के राजपथ पर बहुत बड़ा कार्यक्रम रखा जाता है, जहाँ झंडारोहड़ और राष्ट्रगान के बाद भारत के राष्ट्रपति के समक्ष इंडिया गेट पर भारतीय सेना द्वारा परेड किया जाता है।"

In [ ]:
prompt = 'Given the context delimited by triple backticks ```{'+ essay + '}```, generate a question.Question:'
input_ids = tokenizer.encode(prompt, return_tensors='pt')
greedy_output = model.generate(input_ids, num_beams=7, no_repeat_ngram_size=4,min_length=5, max_length=100, do_sample=True)
print(len(greedy_output),tokenizer.decode(greedy_output[0], skip_special_tokens=True))
question = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
prompt2 = 'Given the context delimited by triple backticks ```{'+ essay + '}```,and the question delimited by single backticks `{'+ question + '}` generate an answer.Answer:'
input_ids = tokenizer.encode(prompt2, return_tensors='pt')
greedy_output = model.generate(input_ids, num_beams=4, no_repeat_ngram_size=2, min_length=20, max_length=100, do_sample=True)
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))